In [ ]:
import pandas_gbq
import pandas as pd
import numpy as np
import db_dtypes

# Read Table

In [ ]:
# load raw table
%%bigquery SLIK --project jago-technical-test
SELECT *
FROM `jago-technical-test.SLIK.SLIK_RAW`

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
# reassign and check columns for convenience
SLIK = SLIK
SLIK.columns

Index(['ktp', 'kreditId', 'posisiDataTerakhir', 'ljk', 'bakiDebet',
       'tanggalUpdate', 'sifatKredit', 'jenisKredit', 'tanggalAkadAwal',
       'tanggalAwalKredit',
       ...
       'tahunBulan23Kol', 'tahunBulan23Ht', 'tahunBulan24', 'tahunBulan24Kol',
       'tahunBulan24Ht', 'akadPembiayaan', 'kategoriDebiturKode',
       'jenisPenggunaanKet', 'jenisAgunan', 'tenor'],
      dtype='object', length=104)

# Data Preparation

## 1. Transform Kol and HT Period

In [ ]:
SLIK.columns

Index(['ktp', 'kreditId', 'posisiDataTerakhir', 'ljk', 'bakiDebet',
       'tanggalUpdate', 'sifatKredit', 'jenisKredit', 'tanggalAkadAwal',
       'tanggalAwalKredit',
       ...
       'tahunBulan23Kol', 'tahunBulan23Ht', 'tahunBulan24', 'tahunBulan24Kol',
       'tahunBulan24Ht', 'akadPembiayaan', 'kategoriDebiturKode',
       'jenisPenggunaanKet', 'jenisAgunan', 'tenor'],
      dtype='object', length=104)

In [ ]:
# list year month of the last 24 period from application date
# list all relevant fields

ym = [202310,202309,202308,202307,202306,202305,202304,202303,202302,202301,
      202212,202211,202210,202209,202208,202207,202206,202205,202204,202203,
      202202,202201,202112,202111]
ym_cols = ['tahunBulan01','tahunBulan02','tahunBulan03','tahunBulan04','tahunBulan05','tahunBulan06',
          'tahunBulan07','tahunBulan08','tahunBulan09','tahunBulan10','tahunBulan11','tahunBulan12',
          'tahunBulan13','tahunBulan14','tahunBulan15','tahunBulan16','tahunBulan17','tahunBulan18',
          'tahunBulan19','tahunBulan20','tahunBulan21','tahunBulan22','tahunBulan23','tahunBulan24']
ym_col_cols = ['tahunBulan01Kol','tahunBulan02Kol','tahunBulan03Kol','tahunBulan04Kol','tahunBulan05Kol','tahunBulan06Kol',
          'tahunBulan07Kol','tahunBulan08Kol','tahunBulan09Kol','tahunBulan10Kol','tahunBulan11Kol','tahunBulan12Kol',
          'tahunBulan13Kol','tahunBulan14Kol','tahunBulan15Kol','tahunBulan16Kol','tahunBulan17Kol','tahunBulan18Kol',
          'tahunBulan19Kol','tahunBulan20Kol','tahunBulan21Kol','tahunBulan22Kol','tahunBulan23Kol','tahunBulan24Kol']
ym_ht_cols = ['tahunBulan01Ht','tahunBulan02Ht','tahunBulan03Ht','tahunBulan04Ht','tahunBulan05Ht','tahunBulan06Ht',
          'tahunBulan07Ht','tahunBulan08Ht','tahunBulan09Ht','tahunBulan10Ht','tahunBulan11Ht','tahunBulan12Ht',
          'tahunBulan13Ht','tahunBulan14Ht','tahunBulan15Ht','tahunBulan16Ht','tahunBulan17Ht','tahunBulan18Ht',
          'tahunBulan19Ht','tahunBulan20Ht','tahunBulan21Ht','tahunBulan22Ht','tahunBulan23Ht','tahunBulan24Ht']

# initialize new cols
for i in ym:
  SLIK[f'Kol_{i}'] = np.nan
  SLIK[f'Ht_{i}'] = np.nan

In [ ]:
# assign value for new cols
for i in list(SLIK.index):
  for j in range(len(ym)):
    yearMonth = ym[j]
    try:
       idx = list(SLIK.loc[i,ym_cols]).index(yearMonth)
       SLIK.loc[i,f'Kol_{yearMonth}'] = SLIK.loc[i,ym_col_cols[idx]]
       SLIK.loc[i,f'Ht_{yearMonth}'] = SLIK.loc[i,ym_ht_cols[idx]]
    except:
      pass

In [ ]:
# fill na
for i in range(len(ym)):
  yearMonth = ym[i]
  SLIK.loc[:,f'Kol_{yearMonth}'] = SLIK[[f'Kol_{yearMonth}']].fillna(0)
  SLIK.loc[:,f'Ht_{yearMonth}'] = SLIK[[f'Ht_{yearMonth}']].fillna(0)

# Export To BQ Table

In [ ]:
# check columns
SLIK.columns

Index(['ktp', 'kreditId', 'posisiDataTerakhir', 'ljk', 'bakiDebet',
       'tanggalUpdate', 'sifatKredit', 'jenisKredit', 'tanggalAkadAwal',
       'tanggalAwalKredit',
       ...
       'Kol_202203', 'Ht_202203', 'Kol_202202', 'Ht_202202', 'Kol_202201',
       'Ht_202201', 'Kol_202112', 'Ht_202112', 'Kol_202111', 'Ht_202111'],
      dtype='object', length=152)

In [ ]:
# export to new table

project_id = 'jago-technical-test'
table_id = 'SLIK.SLIK_COLHT'
SLIK.to_gbq(table_id, project_id=project_id)

<ipython-input-10-1e20f180b513>:3: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  SLIK.to_gbq(table_id, project_id=project_id)


100%|██████████| 1/1 [00:00<00:00, 9915.61it/s]
